# Lab 6: Linear Quadratic Regulator (LQR)

ME C231A, EECS C220B, UC Berkeley

***

In *this* lab we will design and implement a receding-horizon linear quadratic regulator for an inverted pendulum system.  

# System Description 
The inverted pendulum system consists of an inverted pendulum mounted on a cart as shown in the figure.  The objective is to balance the inverted pendulum through the acceleration of the cart.  The dynamics of the inverted pendulum are

\begin{align}
(I + ml^2) \ddot{\theta} + b \dot{\theta} - mgl \sin\theta = ml \cos \theta u.
\end{align}

where $u$ is the cart acceleration, $\theta$ is the pendulum angle and $m$, $l$, and $I$ are the pendulum mass, half-length, and moment of inertia respectively.  See the figure below.

In non-linear state-space form the dynamics are
\begin{align}
\begin{bmatrix} \dot{x}_1 \\ \dot{x}_2 \end{bmatrix} =
\begin{bmatrix} x_2 \\ -a_0 \sin x_1 - a_1 x_2 \end{bmatrix} +
\begin{bmatrix}0 \\ b_0 u\cos x_1 \end{bmatrix}
\end{align}

where $x_1 = \theta$ and $x_2 = \dot{\theta}$, and $a_0 = -\tfrac{mgl}{I + ml^2}$, $a_1 = \tfrac{b}{I + ml^2}$, and  $b_0 = \tfrac{ml}{I + ml^2}$.  In this lab we will use the parameters: $a_0 = -2.5$, $a_1 = 0.05$, and $b_0 = 0.25$.

<center><img src='Pic_InvertedPendulum.png' width="400"/> </center>

In [1]:
from __future__ import division, print_function
import numpy as np

def continuous_dyn(t, x, u, a0, a1, b0):
    theta, thetadot = x
    return [thetadot, - a0*np.sin(theta) - a1*thetadot + b0*u*np.cos(theta)]

def continuous_dyn_with_controller(t, x, Klqr, a0, a1, b0):
    theta, thetadot = x
    k1, k2 = Klqr
    u = -k1*theta-k2*thetadot
    return [thetadot, - a0*np.sin(theta) - a1*thetadot + b0*u*np.cos(theta)]

# Model Construction
We are going to use basic commands to design a linear discrete-time controller to control the non-linear continuous-time inverted pendulum system.  First we linearize the system dynamics about the equilibrium $\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}$ which produces the linearized dynamics

\begin{align}
\begin{bmatrix} \dot{x}_1 \\ \dot{x}_2 \end{bmatrix} =
\underbrace{\begin{bmatrix} 0 & 1 \\ -a_0 & - a_1 \end{bmatrix}}_{A_c}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} +
\underbrace{\begin{bmatrix}0 \\ b_0 \end{bmatrix}}_{B_c} u.
\end{align}

where $\sin(x_1) \approx x_1$ and $\cos(x_1) \approx 1$. Create the appropriate matrices $\texttt{Ac}$ and $\texttt{Bc}$.
Now we need to convert the continuous-time dynamics to discrete-time. This can be accomplished using the $\texttt{scipy}$ command $\texttt{cont2discrete}$

In [2]:
import scipy.signal
Ts = 0.1       # Ts is the discrete sample-time. 
# define the parameters
a0 = -2.5
a1 = 0.05
b0 = 0.25

Ac = np.array([[0, 1],
               [-a0, -a1]])
Bc = np.array([[0],
               [b0]])
# we don't need C and D here. This is just for calling the function
Cc = np.zeros((1,2))
Dc = np.zeros((1,1))
system = (Ac, Bc, Cc, Dc)
Ad, Bd, Cd, Dd, dt = scipy.signal.cont2discrete(system, Ts)

Next use the $\texttt{scipy}$ command $\texttt{solve_discrete_are}$ to create a discrete-time linear quadratic regulator for the discretized system

# Linear Quadratic Regulator

In [3]:
Q = np.diag([1,0])
R = 1

This choice of $Q$ and $R$ represents an infinite-horizon cost of the form
\begin{align}
\sum_{k=0}^{\infty} x_1(k)^2 + u(k)^2
\end{align}

The command to obtain the optimal feedback, $u(k) = -K_{\rm lqr} x(k)$ is below

In [4]:
import scipy.linalg

def dlqr(A, B, Q, R):
    # solve Discrete Algebraic Riccatti equation  
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)

    # compute the LQR gain
    K = scipy.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)

    # stability check 
    eigVals, eigVecs = scipy.linalg.eig(A - B @ K)
    return K, P, eigVals

Klqr, P, eigvals = dlqr(Ad, Bd, Q, R)

# Finally we simulate the inverted pendulum system in closed-loop with the controller.  

In [5]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

x0 = np.array([np.pi/4, 0])
Tf = 10
traj_time = np.arange(0.0, Tf, Ts)
t0 = 0.0

sol = solve_ivp(fun=continuous_dyn_with_controller, t_span=[t0, Tf], t_eval=traj_time, 
                y0=x0, method='RK45', vectorized=True, args=(Klqr.flatten(), a0, a1, b0))

u = [-Klqr.dot(sol.y[:, i]) for i in range(len(sol.y[0,:]))]

Plot the results of the simulation.

In [6]:
%matplotlib widget        

# When using inline plotting, if the figures don't appear, just run the cell separately and do not run all the cells at the same time to fix the issue. 

plt.subplot(3,1,1)
plt.plot(sol.y[0,:])
plt.ylabel('theta')
plt.subplot(3,1,2)
plt.plot(sol.y[1,:])
plt.ylabel('theta_dot')
plt.subplot(3,1,3)
plt.plot(u, 'r')
plt.ylabel('u')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Now repeat the simulation with different values of the weighting matrices $\texttt{Q}$ and $\texttt{R}$.
Try 
* $\texttt{Q = np.diag([1, 2])}$ and $\texttt{R=1}$ 
* $\texttt{Q = np.diag([1, 0])}$ and $\texttt{R=1/100}$ 
* $\texttt{Q = np.diag([1, 10])}$ and $\texttt{R=1/1000}$

How do the different weighting values change the behavior of the closed-loop system?

In [7]:
# Write your code here: